# Part 4 - Platforms Analysis

In [0]:
spark

sc = spark.sparkContext

In [0]:
filepath = "/Users/c.maulard@gmail.com/SteamProject/Steam_data_dataset_zoomed_1.json"

df = spark.read.json(filepath)

In [0]:
from pyspark.sql import functions as F # This will load the class where spark sql functions are contained
from pyspark.sql import Row # this will let us manipulate rows with spark sql
from functools import reduce
from pyspark.sql.functions import when

In [0]:
col_to_drop = ['tags', 'header_image', 'short_description', 'categories', 'discount', 'initialprice', 'languages', 'website', 'required_age', 'release_date']

def dropCol(df, col_name):
  return df.drop(*col_name)

df_platforms = dropCol(df, col_to_drop)
df_platforms.limit(5).toPandas()

,appid,ccu,developer,genre,name,negative,owners,platforms,positive,price,publisher,type
0,10,13990,Valve,Action,Counter-Strike,5199,"10,000,000 .. 20,000,000","{'linux': True, 'mac': True, 'windows': True}",201215,999,Valve,game
1,1000000,0,IndigoBlue Game Studio,"Action, Adventure, Indie",ASCENXION,5,"0 .. 20,000","{'linux': False, 'mac': False, 'windows': True}",27,999,PsychoFlux Entertainment,game
2,1000010,99,NEXT Studios,"Adventure, Indie, RPG, Strategy",Crown Trick,646,"200,000 .. 500,000","{'linux': False, 'mac': False, 'windows': True}",4032,599,"Team17, NEXT Studios",game
3,1000030,76,Vertigo Gaming Inc.,"Action, Indie, Simulation, Strategy","Cook, Serve, Delicious! 3?!",115,"100,000 .. 200,000","{'linux': False, 'mac': True, 'windows': True}",1575,1999,Vertigo Gaming Inc.,game
4,1000040,0,DoubleC Games,"Action, Casual, Indie, Simulation",细胞战争,1,"0 .. 20,000","{'linux': False, 'mac': False, 'windows': True}",0,199,DoubleC Games,game


In [0]:
df_platforms.printSchema()

root
 |-- appid: long (nullable = true)
 |-- ccu: long (nullable = true)
 |-- developer: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- name: string (nullable = true)
 |-- negative: long (nullable = true)
 |-- owners: string (nullable = true)
 |-- platforms: struct (nullable = true)
 |    |-- linux: boolean (nullable = true)
 |    |-- mac: boolean (nullable = true)
 |    |-- windows: boolean (nullable = true)
 |-- positive: long (nullable = true)
 |-- price: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- type: string (nullable = true)



On remarque des nested schemas dans la colonne platforms

In [0]:
from pyspark.sql.functions import array, when, lit #import array pour créer array/listes, when pour les conditions et lit pour créer des valeurs litérales

#put the platforms available i a list
df_platforms = df_platforms.withColumn(
    "platforms_list",
    array(
        when(df_platforms.platforms.windows, lit("Windows")),
        when(df_platforms.platforms.mac, lit("Mac")),
        when(df_platforms.platforms.linux, lit("Linux"))
    )
)

In [0]:
from pyspark.sql.functions import expr #permet d'exécuter du SQL brut dans pyspark

#There is no NONE or null value in the list if not avalaible
df_platforms = df_platforms.withColumn(
    "platforms_list",
    expr("filter(array(" +
         "CASE WHEN platforms.windows THEN 'Windows' END, " +
         "CASE WHEN platforms.mac THEN 'Mac' END, " +
         "CASE WHEN platforms.linux THEN 'Linux' END), " +
         "x -> x IS NOT NULL)")
)

df_platforms.limit(10).display()

appid,ccu,developer,genre,name,negative,owners,platforms,positive,price,publisher,type,platforms_list
10,13990,Valve,Action,Counter-Strike,5199,"10,000,000 .. 20,000,000","List(true, true, true)",201215,999,Valve,game,"List(Windows, Mac, Linux)"
1000000,0,IndigoBlue Game Studio,"Action, Adventure, Indie",ASCENXION,5,"0 .. 20,000","List(false, false, true)",27,999,PsychoFlux Entertainment,game,List(Windows)
1000010,99,NEXT Studios,"Adventure, Indie, RPG, Strategy",Crown Trick,646,"200,000 .. 500,000","List(false, false, true)",4032,599,"Team17, NEXT Studios",game,List(Windows)
1000030,76,Vertigo Gaming Inc.,"Action, Indie, Simulation, Strategy","Cook, Serve, Delicious! 3?!",115,"100,000 .. 200,000","List(false, true, true)",1575,1999,Vertigo Gaming Inc.,game,"List(Windows, Mac)"
1000040,0,DoubleC Games,"Action, Casual, Indie, Simulation",细胞战争,1,"0 .. 20,000","List(false, false, true)",0,199,DoubleC Games,game,List(Windows)
1000080,3,IndieLeague Studio,"Action, Adventure, Indie, RPG",Zengeon,462,"100,000 .. 200,000","List(false, true, true)",1018,799,2P Games,game,"List(Windows, Mac)"
1000100,0,七月九日,"Adventure, Indie, RPG, Strategy",干支セトラ 陽ノ卷｜干支etc. 陽之卷,6,"0 .. 20,000","List(false, false, true)",18,1299,Starship Studio,game,List(Windows)
1000110,0,重庆环游者网络科技,"Action, Adventure, Casual, Free to Play, Massively Multiplayer",Jumping Master(跳跳大咖),34,"20,000 .. 50,000","List(false, false, true)",50,0,重庆环游者网络科技,game,List(Windows)
1000130,0,Simon Codrington,"Casual, Indie",Cube Defender,0,"0 .. 20,000","List(false, true, true)",6,299,Simon Codrington,game,"List(Windows, Mac)"
1000280,0,Villain Role,"Indie, RPG",Tower of Origin2-Worm's Nest,12,"0 .. 20,000","List(false, false, true)",32,1399,Villain Role,game,List(Windows)


### Are most games available on Windows/Mac/Linux instead?



In [0]:
df_platforms_games = df_platforms.select('name', F.explode('platforms_list').alias('platforms'))

df_platforms_games.limit(5).display()

name,platforms
Counter-Strike,Windows
Counter-Strike,Mac
Counter-Strike,Linux
ASCENXION,Windows
Crown Trick,Windows


In [0]:
game_count_platforms = df_platforms_games.groupBy('platforms').agg(F.collect_list('name').alias('games_list'))\
  .withColumn("gameCount", F.size("games_list"))\
  .orderBy(F.desc("gameCount"))

display(game_count_platforms)

platforms games_list gameCount Windows List(Counter-Strike, ASCENXION, Crown Trick, Cook, Serve, Delicious! 3?!, 细胞战争, Zengeon, 干支セトラ　陽ノ卷｜干支etc.　陽之卷, Jumping Master(跳跳大咖), Cube Defender, Tower of Origin2-Worm's Nest, 人气动漫大乱斗, Hellish Quart, SurReal Subway, Rogue Reaper, WRATH: Aeon of Ruin, Drawngeon: Dungeons of Ink and Paper, Battle Motion, The Undisputables : Online Multiplayer Shooter, The Marvellous Machine, Tactical Control, Love Hentai: Endgame, The ScreaMaze, Clam Man, Fist Of Heaven & Hell, Rotund Rebound, Urban Tale, Flex Apocalypse Racing, RogueCraft Squadron, Gachi Finder 3000, Allspace, On Target VR Darts, REPTOMOM, Zero spring episode 3, Kill Tiger, Gun Beat, 雪·疑 Sknow, Glorious Companions, Koi, Kooring VR Wonderland:Mecadino's Attack, Angry Birds VR: Isle of Pigs, Master Of Secrets: Dark Europe, ROWROW, Partial Control, VR Flight Simulator New York - Cessna, SNAILS, VireFit, Next Stop 3, The Town, Tower Behind the Moon, Chronicon Apocalyptica, In Orbit, GUIDE, Grab Lab, Diggerman, Where the Bees Make Honey, The Demon Crystal, Mars Flight VR, Twisting Mower, Diamonds, Xtreme League, KUNAI, Virus Expansion, Casual Desktop Game, RoboVirus, aMAZE Valentine, Fused, There Was A Dream, Zibbs - Alien Survival, Rhythmy, Archeo: Shinar, WARED, Rag Doll Kung Fu, SPITLINGS, INTERPOINT, Hentai beautiful girls, Negligee: Love Stories (all ages), AUSSIE BATTLER TANKS, Vasilis, 孤岛(Isolated Island), War of Three Kingdoms, PiiSim, The Far Kingdoms: Elements, 某1种青春, The Childs Sight, バグダス - デバッガー検定 -, The Five Cores Remastered, Agartha, Victory Road, Fantastic Creatures, Condors Vs Ocelots, Roulette Simulator 2, M.C.I. Escapes, The Spell - A Kinetic Novel, Eggys Games Flash Collection, HA/CK, Tiny Snow, Cell Defender, N-GON, Temple of Pizza, AWAKE - Definitive Edition, Puttin' Around, SOF - RAIDERS, Hand of Horzasha, Ordo Et Chao: New World, Tsumi, Latte Stand Tycoon, 古战三国 Ancient War: Three Kingdoms, Magus Over Fool, Galaxity, Twisty Puzzle Simulator, The Last Dinner, War of Tanks: Blitzkrieg, Gravity Panda, Hentai NetWalk, Freefall 3050AD, Symploke: Legend of Gustavo Bueno (Chapter 3), Punishment Darkness Online, Clicker Warriors, Alluna and Brie, Groomer, Through the Darkest of Times, Hitchhiker - A Mystery Game, TRANSIT, Alpacapaca Double Dash, AIRA VR, Nya Nya Nya Girls 2 (ʻʻʻ)_(=^･ω･^=)_(ʻʻʻ), ШХД: ЗИМА / IT'S WINTER, Graywalkers: Purgatory, The Lord of the Rings: Journeys in Middle-earth, Terrorarium, Fireboy & Watergirl: Elements, cat notebook, Hentai Crush, 学院英雄梦 HeroDreamOfSchool, Crumbling World, Tetris Effect: Connected, NashBored, Abandonment, Vacation Adventures: Park Ranger, CrossTrix, Soko Loco Deluxe, 她2 : 我还想再见到你 Her2 : I Want To See You Again, Aether Way, Dark Sun Pictures' Dark Sun - The Space Shooter, Gravity Ace, Blacksad: Under the Skin, Invasion Zero, De Profundis, Silo 2, Card Brawl, Rhythm Overdrive, time of the zombies, Chaos Caves, Madorica Real Estate, The Truck Game, Boundary VR, Flowers in Dark, Biathlon Battle VR, Pandamonia 潘德莫尼亚, The Revenge of Johnny Bonasera: Episode 3, Hentai Girl Karen, My Island, Maze 3D, Sick Love - An RPG Maker Novel, My Exercise, Area Cooperation Economic Simulation: North Korea (ACES), Paper Cut Mansion, Tools Up!, The Chronicles of Jonah and the Whale, DownStream: VR Whitewater Kayaking, Stickman Racer Road Draw 2, Everpath, blocks, Find-Life EP1, Roombo: First Blood, KungFu Kickball, Unlock Me, POCKET CAR : VRGROUND, iDancer, WRC 8 FIA World Rally Championship, Maiden and Spell, Noel's Hope, PsyOps Solutions, The Secret Order 7: Shadow Breach, Argonauts Agency: Golden Fleece, Fabulous - Angela's True Colors, Fabulous - New York to LA, Yukinas Diary, Delicious - Emily's Road Trip, Pixel Puzzles Illustrations & Anime, Arkane Rush Multiverse Mayhem, The Jackbox Party Pack 6, Snake vs Snake, Myths and Legends - Card Game, Anti-Grav Bamboo-copter, Star Drift Evolution, Freedom Finger, Vision Soft Reset, NationWar2:Chronicle, ELISEISK 2074, The Deep Diving of FloodDragon, Clu

Databricks visualization. Run in Databricks to view.

Windows is top1

### Do certain genres tend to be preferentially available on certain platforms?
### 


In [0]:
df_platforms = df_platforms.withColumn("genre",F.split(df_platforms["genre"], ", "))
df_platforms_genres = df_platforms.select('name', 'platforms_list', F.explode('genre').alias('genre'))
df_platforms_genres = df_platforms_genres.select('name', 'genre', F.explode('platforms_list').alias('platforms'))
display(df_platforms_genres.limit(10))

name,genre,platforms
Counter-Strike,Action,Windows
Counter-Strike,Action,Mac
Counter-Strike,Action,Linux
ASCENXION,Action,Windows
ASCENXION,Adventure,Windows
ASCENXION,Indie,Windows
Crown Trick,Adventure,Windows
Crown Trick,Indie,Windows
Crown Trick,RPG,Windows
Crown Trick,Strategy,Windows


In [0]:
platforms_genres = df_platforms_genres.groupBy("platforms", "genre").count().withColumnRenamed('count', 'genres_counted_per_platform').orderBy(F.desc('genres_counted_per_platform'))

platforms_genres.display()


platforms,genre,genres_counted_per_platform
Windows,Indie,39676
Windows,Action,23755
Windows,Casual,22082
Windows,Adventure,21427
Windows,Strategy,10892
Windows,Simulation,10832
Mac,Indie,9935
Windows,RPG,9533
Linux,Indie,6978
Windows,Early Access,6145


In [0]:
from pyspark.sql.window import Window

w = Window.partitionBy("genre").orderBy(F.desc("genres_counted_per_platform"))
platforms_genres_ranked = platforms_genres.withColumn("rank", F.rank().over(w))
top_platform_per_genre = platforms_genres_ranked.filter(F.col("rank") == 1).drop("rank")

top_platform_per_genre.orderBy(F.desc("genres_counted_per_platform")).limit(10).display()

platforms,genre,genres_counted_per_platform
Windows,Indie,39676
Windows,Action,23755
Windows,Casual,22082
Windows,Adventure,21427
Windows,Strategy,10892
Windows,Simulation,10832
Windows,RPG,9533
Windows,Early Access,6145
Windows,Free to Play,3391
Windows,Sports,2665


Databricks visualization. Run in Databricks to view.

Windows Top 1 again